In [0]:
# imports
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, BooleanType, DecimalType, FloatType,TimestampType
from pyspark.sql.functions import unix_timestamp, col, current_timestamp,hour, dayofweek, date_format
import pandas as pd

In [0]:
# setup parameters
dbutils.widgets.text("product", "Tokyo Tidbits", "Products")
dbutils.widgets.multiselect(name="category", defaultValue="Tokyo Tidbits", choices=["Tokyo Tidbits", "Seattle Sprouts", "Portland Pastries"])

product_param = dbutils.widgets.get("product")

if not product_param:
    product_param = "Tokyo Tidbits"
print(product_param)

Tokyo Tidbits


In [0]:

spark.sql("DROP TABLE IF EXISTS gold_suppliers")


DataFrame[]

In [0]:
%sql
select * from samples.bakehouse.sales_suppliers

supplierID,name,ingredient,continent,city,district,size,longitude,latitude,approved
4000000,Cacao Wonders,cacao,South America,Guayaquil,Las Peñas,M,-79.8974,-2.1791,Y
4000001,Coconut Grove,coconut,Asia,Manila,Intramuros,S,121.0221,14.6042,Y
4000002,Almond Delights,almonds,Europe,Valencia,Ruzafa,L,-0.3762,39.4699,Y
4000003,Sugar Cane Harvest,cane sugar,South America,Sao Paulo,Vila Madalena,XL,-46.6333,-23.5489,Y
4000004,Vanilla Valley,vanilla,North America,Mexico City,Roma Norte,M,-99.1332,19.4326,Y
4000005,Pecan Pleasures,pecans,North America,Atlanta,Virginia-Highland,S,-84.3888,33.749,Y
4000006,Hazelnut Haven,hazelnuts,Europe,Istanbul,Kadıköy,XXL,28.9784,41.0082,Y
4000007,Cinnamon Spice,cinnamon,Asia,Colombo,Galle Face Green,L,79.8612,6.9271,Y
4000008,Cashew Corner,cashews,Asia,Goa,Anjuna Beach,XL,73.8067,15.3173,Y
4000009,Maple Monarch,maple syrup,North America,Montreal,Plateau Mont-Royal,M,-73.5673,45.5017,Y


In [0]:
# helper functions
def add_prefix(df, prefix):
    for col in df.columns:
        df = df.withColumnRenamed(col, prefix + col)

    return df

In [0]:
# load data
suppliers = add_prefix(spark.sql("select * from samples.bakehouse.sales_suppliers"),"sup_")
franchise = add_prefix(spark.sql("select * from samples.bakehouse.sales_franchises"),"franchise_")
transactions = spark.sql("select * from samples.bakehouse.sales_transactions")

In [0]:
joined_data = transactions \
            .join(franchise, transactions.franchiseID == franchise.franchise_franchiseID, "inner") \
            .join(suppliers, franchise.franchise_supplierID == suppliers.sup_supplierID, "inner") \
            .select(
                transactions["customerID"],
                transactions["dateTime"],
                transactions["product"],
                transactions["quantity"],
                transactions["unitPrice"],
                transactions["totalPrice"],
                transactions["paymentMethod"],
                transactions["cardNumber"],
                suppliers["sup_name"],
                suppliers["sup_city"],
                suppliers["sup_approved"]
            )
display(joined_data)

customerID,dateTime,product,quantity,unitPrice,totalPrice,paymentMethod,cardNumber,sup_name,sup_city,sup_approved
1000066,2024-05-09T07:34:29.756Z,Golden Gate Ginger,1,3,3,mastercard,2662172855432474,Molasses Mills,Havana,Y
1000073,2024-05-05T07:47:37.069Z,Tokyo Tidbits,1,3,3,mastercard,2299022983474859,Molasses Mills,Havana,Y
1000059,2024-05-13T09:06:07.464Z,Golden Gate Ginger,1,3,3,amex,348322972857550,Molasses Mills,Havana,Y
1000208,2024-05-05T21:30:24.097Z,Pearly Pies,1,3,3,mastercard,2438051989472114,Molasses Mills,Havana,Y
1000186,2024-05-15T05:14:33.713Z,Outback Oatmeal,1,3,3,mastercard,2640923489914262,Molasses Mills,Havana,Y
1000142,2024-05-04T16:19:32.775Z,Austin Almond Biscotti,1,3,3,visa,4672830672567772,Molasses Mills,Havana,Y
1000260,2024-05-10T10:24:56.855Z,Outback Oatmeal,1,3,3,mastercard,5177762742702070,Molasses Mills,Havana,Y
1000105,2024-05-03T22:39:02.421Z,Tokyo Tidbits,1,3,3,mastercard,2224751615852997,Molasses Mills,Havana,Y
1000234,2024-05-16T21:56:11.134Z,Orchard Oasis,1,3,3,amex,348910581883148,Molasses Mills,Havana,Y
1000112,2024-05-01T05:22:16.193Z,Pearly Pies,1,3,3,amex,345299353548607,Molasses Mills,Havana,Y


In [0]:
dbutils.notebook.exit("success")